## 1 Линейный классификатор

Технологии компьютерного зрения

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_Classifier-1.jpg" width="600">

Как выяснилось (Из разбора домашнего задания, на cifar10 получили невысокую точность точность, необходимо ее повышать), лучше работает метрика L1. Она представляет из себя сумму абсолютных разностей между пикселями. Если сравнивать ее с L2 и визуализировать получившиеся границы классов, видно, что они немного различаются. L1 дает более параллельные границы классов осям координат, в то время как у L2 границы получаются более плавными. С этими метриками мы будем сталкиваться часто: и в качестве loss функции, и в качестве регуляризации, поэтому познакомиться с ними полезно. 

### Алгоритм k-nearest neighbors


<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-KNN.jpg" width="900">

Метод ближайших соседей, в принципе, можно использовать, чтобы создать некоторую очень простую модель, к которой можно обращаться так же, как к сложной. Но чтобы было понятно, что внутри находится обычный питоновский код. В верхней части слайда показан пример кода, как можно все записать в одну строчку. Но проблема заключается в следующем: даже если бы этот код работал более точно, он все равно бы был очень медленным, потому что для определения принадлежности картинки к какому-то классу мы должны сравнить ее со всеми элементами обучающего набора, потому что мы его весь запомнили. Это будет занимать очень много времени, поэтому на практике для классификации такой подход неприемлем. Мы хотим, чтобы обученная модель работала быстро.

Тем не менее, метод ближайшего соседа используется в других задачах, где без него обойтись сложно. Например, в задаче распознавания лиц, когда у нас есть большая база данных, где каждому лицу сопоставлен некий вектор признаков. Для того, чтобы понять, есть ли в нашей базе данных лицо человека, который прошел перед камерой, нужно сделать ровно то же самое, что мы делали с изображениями: нужно его сравнить со всеми элементами в этой базе данных. При этом там нет никакого класса, мы будем смотреть по принципу “похож-не похож”. А если фотографий в базе данных несколько, как на примере сверху, где есть фотографии актеров и некоторое количество векторов в виде точек. Если из семи соседей 5 - это фотографии Джеки Чана, то скорее всего, это его фотография. В таких случаях данный метод вполне полезен.

Ссылки с примерами эффективной реализации метода:

https://github.com/facebookresearch/faiss

<a href="https://arxiv.org/abs/1603.09320">HNSW</a>



## 2 Переобучение и кроссвалидация 

### 2.1 Переобучение

В случае с методом K-Nearest neighbours мы уже столкнулись с тем, что оценивать качество алгоритма на основе выборки, на которой и учился - плохая идея.










In [ ]:
import sklearn.datasets
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 


sns.set_style("whitegrid")

Загрузим датасет с образцами здоровой и раковой ткани. Надо отличать одни от других по набору признаков.

In [ ]:
cancer = sklearn.datasets.load_breast_cancer()
X = cancer.data
y = cancer.target

In [ ]:
cancer.target_names

In [ ]:
X_norm = StandardScaler().fit_transform(X) # normalize data to make all features 'equal'

n_nei_rng = np.arange(1, 30)                                                      # Количество соседей
quality = np.zeros(n_nei_rng.shape[0])

for ind in range(n_nei_rng.shape[0]):
    knn = KNeighborsClassifier(n_neighbors=n_nei_rng[ind])                        # Обучение для разного числа соседей
    knn.fit(X_norm, y)
    q = accuracy_score(y_pred=knn.predict(X_norm), y_true = y)
    quality[ind] = q                                                              # Запоминаем результат обучения в список

plt.figure(figsize=(8,8))
plt.title("KNN on train", size=20)
plt.plot(n_nei_rng, quality)
plt.xticks(n_nei_rng)
plt.show()

Видим, что качество на 1 соседе - самое лучшее. Но это и понятно - ближайшим соседом элемента из обучающей выборки будет сам объект. Мы просто **запомнили** все объекты

На новом объекте ошибка может быть катастрофической

Поэтому выборку всегда делят на обучение и тест 

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=777)
#scaler = StandardScaler()                                                        # Важность выбора метода скалирования !!!
scaler = MinMaxScaler()
#scaler = RobustScaler()
scaler.fit(X_train)
X_train_norm = scaler.transform(X_train)
X_test_norm = scaler.transform(X_test)
#X_train_norm = X_train                                                          # Важность нормирования и стандартизации !!!
#X_test_norm = X_test

n_nei_rng = np.arange(1, 30)
train_quality = np.zeros(n_nei_rng.shape[0])
test_quality = np.zeros(n_nei_rng.shape[0])

for ind in range(n_nei_rng.shape[0]):
    knn = KNeighborsClassifier(n_neighbors=n_nei_rng[ind])
    knn.fit(X_train_norm, y_train)
  
    trq = accuracy_score(y_pred=knn.predict(X_train_norm), y_true = y_train)
    train_quality[ind] = trq

    teq = accuracy_score(y_pred=knn.predict(X_test_norm), y_true = y_test)
    test_quality[ind] = teq

plt.figure(figsize=(8,8))
plt.title("KNN on train vs test", size=20)
plt.plot(n_nei_rng, train_quality, label="train")
plt.plot(n_nei_rng, test_quality, label="test")
plt.legend()
plt.xticks(n_nei_rng)
plt.xlabel("k")
plt.ylabel("Accuracy")
plt.show()

Вот, теперь мы видим, что 1 сосед был "ложной тревогой", надо выбирать число соседей начиная минимум с 3 

### 2.2. Кросс-валидация

Каждая модель имеет как ряд **параметров**, которые она оценивает на основе обучающей выборки, так и ряд **гиперпараметров**, которые влияют на то, каким способом модель будет оценивать свои параметры по выборке. 

В случае KNN параметры, строго говоря, отсутствует - модель просто запоминает объекты обучающей выборки. Особо упорные могут считать их параметрами. 

А вот гиперпараметры есть, даже несколько групп. Какие? 

1. число соседей 
2. функция, которой считаем расстояние между объектами (L2=eucledian, L1=manhattan)
3. веса, с которыми складываем метки ближайших соседей
4. ...
5. признаки! (но об этом с вами поговорим позже)
6. сама модель - мы могли выбрать не KNN, а нагуглить что-нибудь другое

Посмотрим еще раз на график, который нарисовали на прошлом шаге. Какое число соседей считать оптимальным? Метрика явно скачет? 

In [ ]:
plt.figure(figsize=(8,8))
plt.title("KNN on train vs test", size=20)
plt.plot(n_nei_rng, train_quality, label="train")
plt.plot(n_nei_rng, test_quality, label="test")
plt.legend()
plt.xticks(n_nei_rng)
plt.show()


Не понятно, насколько результат зависит от того, как нам повезло или не повезло с разбиением данных на обучение и тест. Может оказаться так, что для конкретного разбиения хорошо выбрать k=5, а для другого - 7. 

Кроме того, опять же - фактически, мы сами выступаем в роли модели, которая учит гиперпараметры (а не параметры) под видимую ей выборку. 

Представим себе, что у нас есть 10000 моделей, полученных подкручиванием разных гиперпараметров (в том числе, выборов просто разного типа модели). Представим, что все эти модели не работают. Вообще. Представим так же, что каждая модель угадывает класс в задаче разделения на два класса с вероятностью 0.5 (и будем считать, что классы у нас сбалансированны - то есть 50% одного класса и 50% другого). 
Опять же, понятно, что таким модели ничем не лучше подбрасывания монетки. 



In [ ]:
def guess_model(y_real):
    guessed = np.random.choice([True, False], 
                               size=y_real.shape[0],
                               replace=True)
    y_predicted = np.zeros_like(y_real)
    y_predicted[guessed] = y_real[guessed]
    y_predicted[~guessed] = 1 - y_real[~guessed]
    return y_predicted

In [ ]:
models_num = 10000
best_quality = 0.5
y_real = np.random.choice([0,1], size=250, replace=True)

for i in range(models_num):
    y_pred = guess_model(y_real)
    q = accuracy_score(y_pred=y_pred, y_true=y_real)
    if q > best_quality:
        best_quality = q
print(best_quality)

То есть мы перебором всех возможных моделей вполне можем получить для абсолютно бесполезной модели приемлемое качество


То есть если подбирать гиперпараметры модели на тесте, то:
1. можно переобучитьcя, просто на более "высоком" уровне. Особенно если гиперпараметров у нее много и все они разнообразны
2. нельзя быть уверенным, что выбор параметров не зависит от разбиения на обучение и тест 

Поэтому мы:

1) подбираем гиперпараметры моделей на отдельном датасете, называмым валидационным. Получаем мы его разбиением обучающего датасета на собственно обучающий и валидационный 

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-cross-validation.png" width="400">

2) чаще всего делаем несколько таких разбиений по какой-то схеме, чтобы получить уверенность оценок качества для моделей с разными гиперпараметрами - кросс-валидация

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-cross-validation2.png" width="500">

Часто применяется следующий подход, называемый K-Fold кросс-валидацией:

https://scikit-learn.org/stable/modules/cross_validation.html


Берется тренировочная часть датасета, разбивается на части - блоки. Дальше мы будем использовать для проверки первую часть (Fold 1), а на остальных учиться. И так последовательно для всех частей. В результате у нас будут информация о точности для разных фрагментов данных и уже на основании этого можно понять, насколько значение этого параметра, который мы проверяем, зависит или не зависит от данных. То есть если у нас от разбиения точность при одном и том же К меняться не будет, значит мы подобрали правильное К. Если она будет сильно меняться в зависимости от того, на каком куске данных мы проводим тестирование, значит, это просто такие данные.  


In [ ]:
from sklearn.model_selection import GridSearchCV, KFold
# Рекомендую прочитать о GridSearchCV, KFold
# можно в русскоявычной интернете

In [ ]:
model = GridSearchCV(KNeighborsClassifier(), 
                     cv = KFold(3, 
                                shuffle=True,
                                random_state=555),
                     param_grid={"n_neighbors": np.arange(1, 20), 
                                 "metric": ["euclidean", "manhattan"],
                                 "weights": ["uniform", "distance"]},
                     scoring="accuracy",
                     n_jobs=-1)
model.fit(X_train_norm, y_train)

In [ ]:
model.best_params_

Вот и список оптимальных параметров с точки зрения кросс-валидации

Объект GridSearchCV можно использовать как обычную модель.

In [ ]:
y_pred = model.predict(X_test_norm)
print(accuracy_score(y_pred=y_pred, y_true=y_test))

Но из него еще можно извлечь дополнительные данные о кроссвалидации

In [ ]:
model.cv_results_['mean_test_score']

In [ ]:
model.cv_results_['std_test_score']

In [ ]:
model.cv_results_['params']

Построим, например, при фиксированных остальных параметрах (равных лучшим параметрам), качество модели на валидации в зависимости от числа соседей

In [ ]:
selected_means = []
selected_std = []
n_nei = []
for ind, params in enumerate(model.cv_results_['params']):
    if params['metric'] == model.best_params_['metric'] and\
       params['weights'] == model.best_params_['weights']:
       n_nei.append(params['n_neighbors'])
       selected_means.append(model.cv_results_['mean_test_score'][ind])
       selected_std.append(model.cv_results_['std_test_score'][ind])

In [ ]:
plt.figure(figsize=(8,8))
plt.title(f"KNN CV, {params['metric']}, {params['weights']}", size=20)
plt.errorbar(n_nei,  selected_means, yerr=selected_std, linestyle='None',fmt='-o')
plt.xticks(n_nei)
plt.show()

Видим, что на самом деле большой разницы в числе соседей и нет. 

### Можно ли делать только кросс-валидацию (без теста)

Нет, нельзя. Кросс-валидация не до конца спасает от подгона параметров модели под выборку, на которой она проводится.

Оценка конечного качества модели должно производиться на отложенной тестовой выборке. 

Если у вас очень мало данных, можно рассмотреть [вложенную кросс-валидацию](https://weina.me/nested-cross-validation/)

Но даже в этом случае надо тогда анализировать поведение вашей модели и тд, чтобы показать, что она учит что-то разумное.

Кстати, вложенную кросс-валидацию можно использовать чтобы просто получить более устойчивю оценку поведения модели на тесте

## 3. Линейный классификатор

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-04.jpg" width="800">

Давайте подумаем, как избавиться от проблемой со скоростью K-Nearest Neighbors. Реализуя метод ближайшего соседа, мы сравнивали одно изображение со всеми, при этом мы предполагали, что изображения из одного класса будут чем-то похожи друг на друга, и поэтому разница будет меньше. Метод как-то работал. Но в целом нам важно понять, для изображения какого класса похоже наше. 

Чтобы ускорить этот процесс, мы можем взять весь блок изображений (справа), который относится, например, к машинам, каким-то образом сольем в один массив (усредним). Таким образом, получим некоторый шаблон для класса “автомобиль”. Возможно, это будет работать не слишком здорово, но зато вместо тысяч изображений (в данном случае 5000) появится одно. Возможно, это поможет сильно сэкономить время.

### 3.1 Переход к сравнению с шаблоном

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-05.jpg" width="450">

То есть, идея в следующем: вместо того, чтобы сравнивать каждое изображение со всеми остальными по очереди, будем сравнивать изображения с шаблоном для каждого класса. Их будет всего 10 для данного датасета. Этот подход позволит радикально увеличить скорость. 

Проверим, что получится из этой идеи. 


скачиваем CIFAR10 в архиве

In [ ]:
!wget https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
!tar -xzf cifar-10-python.tar.gz
!ls -l

Загрузили архив в пямять целиком, используя для этого фрагмент кода с сайта CIFAR10. 

In [ ]:
import numpy as np

def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

x_train = np.zeros((0,3072))
y_train = np.array([])
for i in range(1,6):
  raw = unpickle(f"/content/cifar-10-batches-py/data_batch_{i}")
  x_train = np.append(x_train,np.array(raw[b'data']),axis=0)
  y_train = np.append(y_train,np.array(raw[b'labels']),axis=0)

test = unpickle("/content/cifar-10-batches-py/test_batch")
x_test = np.array(test[b'data'])
y_test = np.array(test[b'labels'])

# Load label names. For for convenience only.
labels_ru = ['Самолет', 'Автомобиль', 'Птица', 'Кошка', 'Олень', 'Собака', 'Лягушка', 'Лошадь', 'Корабль', 'Грузовик']

print(x_train.shape)
print(x_test.shape)

Код классификатора. Идея полностью аналогична NN, но сравнение идет с шаблоном, который генерируется во время обучения.

In [ ]:
class TemplateBasedClassifier:
  def __init__(self):
    self.templates = []
    meta = unpickle("/content/cifar-10-batches-py/batches.meta")
    self.labels= meta[b'label_names']

  def fit(self, x_train, y_train):
    self.templates = []
    for label_num in range(len(self.labels)):
      indexes = np.where(y_train == label_num)
      mask = np.zeros(len(y_train), dtype=bool)
      mask[indexes,] = True
      images = x_train[mask]
      mn = np.mean(images,0)
      self.templates.append(mn)
      
  def forward(self,x):
    distances = np.sum(np.abs(self.templates - x),axis =1)
    return np.argmin(distances)

Проверим, насколько поменялся результат.

In [ ]:
import time
def validate(model,x_test,y_test,noprint=False):
  correct = 0
  for i,img in enumerate(x_test):
    index = model.forward(img)
    correct += 1 if index == y_test[i] else 0
    if noprint is False:
      if i > 0 and i % 1000 == 0:
        print ("Accuracy {:.3f}".format(correct/i))
  return correct/len(y_test)  

model = TemplateBasedClassifier()
start = time.perf_counter()
model.fit(x_train,y_train)
print("Train time",time.perf_counter() - start)
start = time.perf_counter()
accuracy = validate(model,x_test,y_test)
tm = time.perf_counter() - start
print("Accuracy {:.2f} Train {:d} /test {:d} in {:.1f} sec. speed {:.2f} samples per second.".format(accuracy,len(x_train),len(x_test),tm,len(x_test)/tm) )

Точность упала незначительно — 0.27. Зато скорость возросла более чем в 600 раз!

Визуализируем шаблоны:

In [ ]:
from PIL import Image
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (20.0, 20.0)

def show_templates(model):
    for i, template in enumerate(model.templates):
      img = (template/max(template))*255
      img = template.reshape(3,32,32).transpose(1,2,0).astype(int)
      plt.subplot(1, len(model.labels), i+1)
      plt.title(model.labels[i])
      imshow(img)

show_templates(model)

Итак, мы придумали, как намного быстрее и эффективнее проводить обучение - для этого мы изготовили специальные шаблоны. В процессе обучения модель стала извлекать из данных какие-то общие черты.
Сейчас мы визуализировали получившиеся шаблоны.

Теперь у нас есть 10 шаблонов, с которыми мы будем сравнивать изображение и на основании этого делать предсказание. На этих шаблонах можно увидеть очертания объектов.


### 3.2 Переход к весам
----
Умножение вместо вычитания - для чего?


<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-06.jpg" width="450">

Теперь давайте сделаем следующий, пока ничем не обоснованный, ход: оставим всё, как было, но заменим вычитание умножением. Логика в том, что не все пиксели одинаково важны. Вероятно, если на изображении совпадут какие-то пиксели, которые отвечают, например, за глаза кошки, это будет намного важнее, чем фон, который может быть точно таким же у собаки. Здесь будут какие-то важные особенности, которым можно придать больший вес.

Если мы распишем в виде формул наложение шаблона на картинку таким образом с умножением, то получается, что мы скалярно перемножаем два вектора. Если один вектор – это картинка, а второй – изображение, если мы их вытянем, у нас получается <a href="https://ru.wikipedia.org/wiki/Скалярное_произведение">скалярное произведение</a> векторов.


### 3.3 Математическая запись

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-08.jpg" width="700">



Обозначим входное изображение как x, а шаблон для первого из классов как $w_0$
Элементы пронумеруем подряд 1,2,3 … $n$
То есть развернем матрицу пикселей изображения в вектор. 

На картинке одноканальное изображение.
Но если каналов несколько, то принципиально это ничего не изменит: мы можем добавлять их в наш вектор один за одним. 
И его длина(n) будет  H*W*C где С - количество каналов.

Для CIFAR10 n =  32*32*3

Тогда результат сравнения изображения с этим шаблоном будет вычисляться по формуле: $x[0]*w0[0] + x[1]*w0[1] + … x[n-1]*w0[n-1]$ 




<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-skalyar.png" width="800">


Этот подход используется очень широко. Во-первых, ровно так работает искусственный нейрон: у него есть набор входов, которые перемножаются на веса и дальше суммируются. Эта простая модель лежит в основе практически всех сложных, которые мы будем рассматривать дальше. То есть внутри мы будем пользоваться скалярным произведением.

Сама свертка работает очень похоже: мы тоже накладываем шаблон на некоторую матрицу и перемножаем элементы, затем складываем. Единственное отличие – обычно ядро свертки меньше, чем размер самого изображения. Этот же принцип использует линейный классификатор, который вы, возможно, использовали раньше, может быть просто не для классификации изображений. С помощью аналогичного выражения задаются гиперплоскости, которые разделяют данные. Простейшим его применением является линейная регрессия, где для случая одной переменной можно увидеть все на координатной плоскости.




### 3.4 Пример простой линейной регрессии

В этой задаче регрессии мы спрогнозируем успеваемость студента, в зависимости от количества часов, которые он учил материал. Это простая задача линейной регрессии, поскольку она включает всего две переменные.

In [ ]:
!wget http://edunet.kea.su/repo/src/L02_Linear_classifier/datasets/student_scores.csv

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
dataset = pd.read_csv('/content/student_scores.csv')
print(dataset.shape)
dataset.head()

In [ ]:
dataset.describe()

In [ ]:
type(dataset)

In [ ]:
%matplotlib inline
dataset.plot(x='Hours', y='Scores', style='o')
plt.title('Hours vs Percentage')
plt.xlabel('Hours Studied')
plt.ylabel('Percentage Score')
plt.show()

In [ ]:
# Preparation data
# Remove labels from dataset
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, 1].values

# Separate data to train and test parts
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
type(X), type(y)

In [ ]:
# Train LinearRegression from sklearn
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

In [ ]:
%matplotlib inline
x_points = np.linspace(min(X_train), max(X_train), 100)
y_pred = regressor.predict(x_points)

dataset.plot(x='Hours', y='Scores', style='o')
plt.plot(x_points, y_pred, label = "Fit line")
plt.title('Hours vs Percentage')
plt.xlabel('Hours Studied')
plt.ylabel('Percentage Score')
plt.legend()
plt.show()

In [ ]:
# Linear equetion is y = coef * x + intercept
print("coef =", regressor.coef_[0])
print("intercept =", regressor.intercept_)

In [ ]:
# Make predictions for test part
y_pred = regressor.predict(X_test)
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

# Show Actual value comparison with Predicted value
df.head()

In [ ]:
type(df)

In [ ]:
# Evaluate model
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

### 3.5 Геометрическая интерпретация 

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-09.jpg" width="750">


http://vision.stanford.edu/teaching/cs231n-demos/linear-classify/


Прямая может не проходить через 0, данные могут быть распределены так, что они будут далеко от нуля. Чтобы модель линейного классификатора работала хорошо, мы должны к нашим данным добавить смещение. Это смещение позволит проводить разделяющие плоскости или гиперплоскости в n-мерном пространстве уже не через 0, а некоторым произвольным образом, в зависимости от того, какие будут данные, на которых они обучились. Если у нас есть несколько классов (несколько шаблонов), мы для каждого получаем уравнение kx+b, соответственно, если пространство n-мерное, k тоже будет n-мерным и для каждого объекта этот вектор будет свой. 






Мы их можем собрать в матрицу, тогда получится следующее: 


### 3.6 Добавление смещения

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-10.jpg" width="800">

У нас есть матрица коэффициентов, которые мы каким-то образом подобрали, пока ещё не понятно как. Есть вектор, соответствующий изображению. Мы умножаем вектор на матрицу, получаем нашу гиперплоскость для четырехмерного пространства в данном случае. Чтобы оно не лежало в 0, мы должны добавить смещение. И мы можем сделать это после, но можно взять и этот вектор смещения (вектор **b**) просто приписать к матрице **W**.

Что будет выходом такой конструкции? Мы умножили матрицу весов на наш вектор, соответствующий изображению, получили некоторый отклик. По этому отклику мы так же, как и при реализации метода ближайшего соседа можем судить: если он больше остальных, то мы предполагаем, что это кошка.


<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-07.jpg" width="800">


Собирая его вместе, получаем какое-то компактное представление, что у нас есть некоторая функция, на вход которой мы подаем изображение, и у него есть параметры (веса). Пока происходит просто умножение вектора на матрицу, в дальнейшем это может быть что-то более сложное, функция будет представлять какую-то более сложную модель. А на выходе ( для классификатора) мы получаем числа, которые интерпретируют уверенность модели в том, что изображение принадлежит к определенному классу.


<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-11.jpg" width="600">



То есть, ничего не поменялось по сравнению с тем, что мы просто сравнивали шаблоны, кроме способа того, как мы их накладываем, то есть менялся знак, добавилось ещё смещение в виде одного числа. На каждый шаблон мы должны ещё запомнить одно число, соответствующее смещению. В целом логика осталась такой же.

Соответственно, эти коэффициенты, которые являются весами модели, надо каким-то образом подбирать. Но прежде, чем подбирать коэффициенты, давайте определимся со следующим: как мы будем понимать, что модель работает хорошо или плохо? Ведь она возвращает достатотчно абстрактные числа, которые нужно уметь интерпретировать.


## 4. Оценка результата

https://ru.wikipedia.org/wiki/Метод_опорных_векторов

### 4.1 Функция потерь SVM-loss

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-12.jpg" width="550">

По аналогии с тем, что мы уже делали, мы можем сравнивать отклик на ключевой класс  (про который нам известно, что он на изображении, так как у нас есть метка этого класса) с остальными. Соответственно, мы подали изображение кошки и получили на выход вектор. Чем больше уверенность, тем больше вероятность того, что, по мнению модели, на изображении этот класс. Для кошки в данном случае это значение 2.9. Хорошо это или плохо? Нельзя сказать, пока мы не проанализировали остальную часть вектора. Если мы двинемся по нему, мы увидим, что здесь есть значения больше, то есть в данном случае модель считает, что это собака, а не кошка, потому что для собаки значение максимально. 

На этом можно построить некоторую оценку. Давайте смотреть разницу правильного класса с неправильными. В зависимости от того, насколько уверенность в кошке будет больше остальных, настолько будет наша модель хорошей.

Но поскольку нам важна не работа модели на конкретном изображении, а важно оценить ее работу в целом, то эту операцию нужно проделать либо для всего датасета, либо для некоторого пакета, который мы подаем на вход и подсчитываем средний показатель. Этот показатель того, как хорошо работает модель, называется функцией потерь, или loss функцией. Называется она так потому, что она показывает не то, насколько хорошо работает модель, а то, насколько плохо. 

Дальше будет понятно, почему так удобнее (разница только в знаке).
Как это посчитать для всего датасета?

Мы каким-то образом считаем ее для конкретного изображения, потом усредняем по всем изображениям.

Дано: 3 учебных примера, 3 класса. При некотором W баллы f (a, W) = Wx равны:

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-13_1.jpg" width="500">

Функция потерь показывает, насколько хорош наш текущий классификатор.

Дан датасет примеров:

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-13_2.png" width="200">

Где xᵢs изображение и yᵢs метка (число).

Потери по набору данных — это среднее значение потерь для примеров:

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-13_3.png" width="350">

Снова возьмём наши обучающие примеры **$x_i$** и метки классов **$y_i$**. Представим прогнозы классификатора (оценки) в виде векторов: **$s = f(xi, W)$**. Для каждого отдельного примера просуммируем значения оценок всех **$y$**, кроме истинной метки **$yi$**. То есть, мы получим сумму значений всех неправильных категорий. 

Теперь посчитаем разницу между ложными прогнозами и истинной оценкой: **$sj − syi$**; **$j≠yi$**. Если истинная оценка больше, чем сумма всех неправильных прогнозов плюс некоторый дополнительный «зазор» (установим его равным 1) — значит, полученный балл для правильной категории намного превосходит любую ошибочную оценку. Это и будет наша функция потерь.

В символьном виде это выглядит так:

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-SVM1.png" width="350">



### 4.2 Вычисление SVM loss


 Не самая мощная, но достаточно интуитивно понятная loss функция –  SVM loss. Она пришла из классического машинного обучения, из метода опорных векторов. 
 
Логика такая: если у нас уверенность модели в правильном классе большая, то модель работает хорошо и loss для данного конкретного примера должен быть равен нулю. Если есть класс, в котором модель уверена больше, чем в правильном, то loss должен быть не равен нулю, а отображать какую-то разницу, поскольку модель сильно ошиблась. При этом есть ещё одно соображение: что будет, если на выходе у правильного и ошибочного класса будут примерно равные веса? То есть, например, у кошки было бы 3.2, а у машины не 5.2, а 3.1. В этом случае ошибки нет, но понятно, что при небольшом изменении в данных (просто шум) скорее всего она появится.
 


То есть модель плохо отличает эти классы. Поэтому можно ввести некоторый зазор, который должен быть между правильным и неправильным ответом. 

<br>
<img src ="https://kodomo.fbb.msu.ru/FBB/year_20/svm_decision_boundary.png" width="550">

И тоже учитывать его в loss функции: то есть сравнивать его результат для правильного класса не с чистым выходом для другого, а добавить к нему некоторую дельту (в данном случае – 1(единица)). Смотрим: если разница больше 0, то модель работает хорошо и loss равно нулю. Если нет, то мы возвращаем эту разницу и loss будет складываться из этих индивидуальных разниц.

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-SVM1.png" width="350">

Ниже пример того, как считается loss.


Считаем функцию потерь для 1го изображения:

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-SVM1.jpg" width="650">

Также считаем потери для 2го и 3его изображения:

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-SVM2_1.jpg" >

Значения losses получились следующие:

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-SVM4.png" width="450">

Считаем среднее значение loss для всего датасета:

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-SVM5.png" width="350">

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-16.jpg" width="450">

Как записать это в коде:

In [ ]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

x_train = np.zeros((0,3072))
y_train = np.array([])
for i in range(1,6):
  raw = unpickle(f"/content/cifar-10-batches-py/data_batch_{i}")
  x_train = np.append(x_train,np.array(raw[b'data']),axis=0)
  y_train = np.append(y_train,np.array(raw[b'labels']),axis=0)

test = unpickle("/content/cifar-10-batches-py/test_batch")
x_test = np.array(test[b'data'])
y_test = np.array(test[b'labels'])

# Load label names. For for convenience only.
labels_ru = ['Самолет', 'Автомобиль', 'Птица', 'Кошка', 'Олень', 'Собака', 'Лягушка', 'Лошадь', 'Корабль', 'Грузовик']

print(x_train.shape)
print(x_test.shape)

In [ ]:
W = np.random.randn(10, 3072) * 0.0001  # For CIFAR10

def Li_svm(x,y,W):
  y = int(y) # for use as index
  scores = W.dot(x)
  margins = np.maximum(0,scores - scores[y] + 1 ) # delta = 1
  margins[y] = 0 # Что бы не писать лишнее условие
  return np.sum(margins)

L0 = Li_svm(x_train[0],y_train[0],W)
print("Loss on first image", L0)

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-15.jpg" width="900">

Собираем все вместе: считаем loss для всего датасета, усредняя ее.


Как будет выглядеть график этой SVM-loss?

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/hinge_loss.gif" width="400">


Функция потерь для ситуации, когда зазор большой, будет равна 0. Она будет меняться только тогда, когда у нас есть ошибка. Причем будет меняться линейно.

Как мы можем использовать эту loss функцию? Допустим, мы знаем, что у нас модель будет работать не очень хорошо (выдает большую ошибку). Можно попытаться найти минимум этой loss функции, то есть подобрать такое W, когда loss функция обратится в 0  (в данном случае), а в общем случае (если какое-то количество данных нельзя обратить в 0), мы можем попытаться найти ее минимум.



## 5. Обновления весов методом градиентного спуска

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-18.png" width="750">

Если переходить на случай n-мерного (в данном случае трехмерного) пространства, здесь достаточно очевидна аналогия с поверхностью: у нас есть поверхность земли, котораяя описывается координатами x, y, z. Если мы движемся по этой поверхности, высота (z) будет зависеть от x, y. 

x, y - это координаты, мы можем записать их как вектор. Наша функция будет работать с вектором координат и будет выдавать скаляр, третью координату. Если в качестве координат мы будем использовать веса нашей модели, а в качестве z - loss, то аналогия станет полной. Наша задача сведется к тому, чтобы найти такой набор весов, при котором значение функции будет минимально. То есть мы окажемся в каком-то минимуме, где ошибка будет минимально возможна для этих данных.

Для того, чтобы предположить, где он может находиться, надо понимать, в какую сторону функция растет, а в какую - убывает. Для этого существует производная.

Поскольку у нас функция от нескольких переменных, если мы будем брать от нее производную, у нас получится вектор частных производных, то есть градиент этой функции, который будет показывать, как она меняется в каждом направлении.

### Градиент loss функции


Градиент — вектор, своим направлением указывающий направление наибольшего возрастания некоторой величины **φ**, значение которой меняется от одной точки пространства к другой (скалярного поля), а по величине (модулю) равный скорости роста этой величины в этом направлении.

Например, если взять в качестве **φ**  высоту поверхности земли над уровнем моря, то её градиент в каждой точке поверхности будет показывать «направление самого крутого подъёма», и своей величиной характеризовать крутизну склона.

Другими словами, градиент — это производная по пространству, но в отличие от производной по одномерному времени, градиент является не скаляром, а векторной величиной.

Для случая трёхмерного пространства градиентом скалярной функции **φ=φ(x,y,z)** координат **(x,y,z)** называется векторная функция с компонентами

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/gradient.svg">


Если **φ**  — функция **n** переменных **X1...xn**, то её градиентом называется **n**-мерный вектор

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/gradient4.svg">

компоненты которого равны частным производным **φ** по всем её аргументам.

Размерность вектора градиента определяется, таким образом, размерностью пространства (или многообразия), на котором задано скалярное поле, о градиенте которого идёт речь.
Оператором градиента называется оператор, действие которого на скалярную функцию (поле) даёт её градиент. Этот оператор иногда коротко называют просто «градиентом».

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-gradient.png" width="650">

Наша задача будет сводиться к тому, что мы будем искать градиент loss функции по весам, которые будут состоять из производной по каждому направлению. Поскольку у нас здесь числа, можно считать этот градиент численно.

### 5.1 Численный расчет производной

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-gradient_spusk.png" width="650">



Посчитаем градиент приближенно, воспользовавшись определением (в формуле аргумент обозначен как х у нас же аргументом будет W):
На нулевом шаге у нас есть $W_0$ найдем $L_0 = Loss(f(W0,x))$
Прибавим к первому элементу $W0[0]$ небольшую величину  $h$ = 0.0001 и получим новую матрицу весов $W_1$ отличающуюся от $W_0$ на один единственный элемент
Найдем Loss от $W_1$ : $L_1 = Loss(f(W_1,x))$
По определению производной $dL/W_0 = ( L_1 - L_0 ) /  h$
Повторяя этот процесс для каждого элемента из W найдем вектор частных производных то есть градиент $dL/dW$.

Плюсы:
Это просто. 

Проблемы:

1. Это очень долго, нам придется заново искать значение loss функции для каждого $W_i$.

2. Это неточно, так как по определению приращение $h$ бесконечно мало, а мы используем конкретное пусть и небольшое число. И если мы сделаем его слишком маленьким то столкнемся с ошибками связанными с округлением в памяти компьютера.


Поэтому данный метод может использоваться как проверочный. 
А на практике вместо него используется **аналитический расчет градиента**.


### 5.2 Аналитический расчет производной от SVM Loss

#### 5.2.1 Простые производные 

$$x' = \frac {\delta x} {\delta x} = 1$$ 

$$(x^2)' = \frac {\delta x^2} {\delta x} = 2x$$

$$(\log x)'  = \frac {\delta \log x} {\delta x} = \frac 1 x $$

$$(e^x)'  = \frac {\delta e^x} {\delta x} = e^x $$

$$\frac {\delta cf(x)} {\delta x}= c \cdot \frac {\delta f(x)} {\delta x}$$

$$\frac {\delta f(x) + c} {\delta x}= \frac {\delta f(x)} {\delta x}$$

c - константа, не зависящая от x

$$ \frac {\delta [f(x) + g(x)]} {\delta x} = \frac {\delta f(x)} {\delta x}  + \frac {\delta g(x)} {\delta x} $$ 


$$\frac {\delta (x^2 + y^3)} {\delta x} = 2x  $$
так как y по отношению к x - константа и мы меняем только x

$$\frac {\delta (x^2 + y^3)} {\delta y} = 3y^2  $$
так как x по отношению к y - константа и мы меняем только y

$$(e^y)'  = \frac {\delta e^y} {\delta x} = 0 $$

#### 5.2.2 Chain-rule

Производная функции $f(g)$:

$$\frac {\delta f} {\delta g}$$

Пусть g  на самом деле не просто переменная, а зависит от h. Тогда производная от f по g **не меняется**, а производная f по h запишется следующим образом:


$$\frac {\delta f(g(h))} {\delta h} = \frac {\delta f} {\delta g} \frac {\delta g} {\delta h}$$

Пусть теперь h зависит от x. Все аналогично


$$\frac {\delta f(g(h(x)))} {\delta x} = \frac {\delta f} {\delta g} \frac {\delta g} {\delta h} \frac {\delta h} {\delta x}$$

Так можно делать до бесконечности, находя производную сколь угодно сложной функции. И, что важно - мы можем считать градиенты частями - посчитать сначала f по g, потом g по h,....

$$\frac {\delta log(x^2 + 5)} {\delta x}$$

$$h = x^2 + 5$$

$$\frac {\delta log(x^2 + 5)} {\delta x} = \frac {\delta log(h)} {\delta h} \frac {\delta h} {\delta x}$$ 

$$\frac {\delta log(h)} {\delta h} = \frac 1 h$$

$$\frac  {\delta h} {\delta x} = 2x $$

$$\frac {\delta log(x^2 + 5)} {\delta x} = \frac 1 {x^2 + 5} \cdot 2x = \frac {2x} {x^2 + 5}$$


#### 5.2.3 Часть MSE-loss

$$loss = (y - \hat{y})^2 $$

$$\hat{y} = wx + b $$

$$ \frac {\delta loss} {\delta w} = \frac {\delta loss} {\delta \hat{y}} \cdot \frac {\delta \hat{y}} {\delta w} $$

$$ \frac {\delta loss} {\delta \hat{y}} = \frac {\delta (y - \hat{y})^2 } {\delta {(y - \hat{y})}} \frac {\delta y - \hat{y}} {\delta \hat{y}} = 2(y-\hat{y}) \cdot -1 = 2 (\hat{y} - y)$$

$$ \frac {\delta \hat{y}} {\delta w} = \frac {\delta wx + b} {\delta w} = x$$

$$ \frac {\delta loss} {\delta w} = 2 x \cdot (\hat{y} - y) $$






#### 5.2.4 MSE-loss
$$MSE = \frac 1 N \sum_i(y_i - \hat{y_i})^2 $$

$y_i$ - константы
$\hat{y_i}$ - не являются функциями друг от друга 
$$\hat{y} = wx_i + b $$

$$\frac {\delta MSE} {\delta w} = \frac 1 N \sum \frac {\delta (y_i - \hat{y_i}) ^2} {\delta \hat{y_i}} \frac {\delta \hat{y_i}} {\delta w}$$


#### 5.2.4 Часть MAE-Loss

$$loss = |y - \hat{y}| $$

$$\hat{y} = wx + b $$

$$ \frac {\delta loss} {\delta w} = \frac {\delta loss} {\delta \hat{y}} \cdot \frac {\delta \hat{y}} {\delta w} $$

$$ \frac {\delta \hat{y}} {\delta w} = \frac {\delta wx + b} {\delta w} = x$$

$$ \frac {\delta loss} {\delta \hat{y}} = \frac {\delta |y - \hat{y}|) } {\delta {(y - \hat{y})}} \frac {\delta y - \hat{y}} {\delta \hat{y}} = \frac {\delta |y - \hat{y}|) } {\delta {(y - \hat{y})}} \cdot -1 = - \frac {\delta |y - \hat{y}|) } {\delta {(y - \hat{y})}}$$

Строго говоря, у модуля не существует производной в 0. 

![altext](https://upload.wikimedia.org/wikipedia/commons/thumb/6/6b/Absolute_value.svg/1200px-Absolute_value.svg.png)

Но мы можем сказать, что в этой точке производная равна 0.
Если аргумент модуля меньше 0, то производная будет -1. Если больше - +1

$$ \frac {\delta loss} {\delta \hat{y}} = \frac {\delta |y - \hat{y}| } {\delta {(y - \hat{y})}} \frac {\delta y - \hat{y}} {\delta \hat{y}} = \frac {\delta |y - \hat{y}| } {\delta {(y - \hat{y})}} \cdot -1 = - \frac {\delta |y - \hat{y}| } {\delta {(y - \hat{y})}} = - sign(y - \hat{y}) =  sign(\hat{y} - y)$$

<br>
<br>

<img src ="https://upload.wikimedia.org/wikipedia/commons/thumb/4/4f/Signum_function.svg/1200px-Signum_function.svg.png" width="500">



#### 5.2.5 2-Max-Loss

$$b = max(x, y)$$

$$b = x~~if~~x > y~~else~~y$$


$$\frac {\delta b} {\delta x} =  \frac {\delta x} {\delta x}~~if~~x > y~~else~~\frac {\delta y} {\delta x} = 1~~if~~x > y~~else~~0$$

Если x > y, то он оказал влияние на b. Иначе его вклада в b НЕ БЫЛО - градиент равен 0


#### 5.2.6 k-Max-Loss

Аналогично. 


#### 5.2.7 SVM-Loss

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier_an_r_g1.png" width="750"> 


Распишем формулу для $L_i$ через скалярное произведение.
Затем раскроем сумму.


<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier_an_r_g2.png" width="750"> 

Считаем производную по первому элементу матрицы **W**. 
Если у нас правильный класс - не первый элемент, то он больше нигде не встретится, кроме как в первой сумме. Поэтому все остальные суммы при подсчете этой частной производной будут константами. Поэтому этот градиент будет равен либо $x_i$,  либо 0 (если эта сумма меньше 0). 

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier_an_r_g3.png" width="750"> 

Аналогично для  $w_i$ частные производные будут равны $x_i$ либо нулю, когда выражение больше меньше 0, 
кроме случая: $j == yi$.  



<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier_an_r_g4.png" width="750"> 


Эта ситуация для “правильного изображения”.



<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier_an_r_g5.png" width="750">  

Вот финальный результат.
Общие потери считаются по пакету изображений.
“1” в формулах  — это “if”.


In [ ]:
import numpy as np
import random

class LinearClassifier:
  def __init__(self, labels, batch_size, random_state=777):
    self.labels = labels
    self.classes_num = len(labels)
    # Generate a random weight matrix of small numbers
    # Number of weights changed from 3072 to 3073 for implement bias trick
    np.random.seed(random_state)
    self.W = np.random.randn(3073, self.classes_num) * 0.0001
    self.batch_size = batch_size #256
    

  def fit(self, x_train, y_train, learning_rate = 1e-8):
    loss = 0.0
    train_len = x_train.shape[0]
    indexes = list(range(train_len))
    random.shuffle(indexes)

    for i in range(0, train_len, self.batch_size):
      # Split data into batches 
      idx = indexes[i:i+self.batch_size]
      x_batch = x_train[idx]
      y_batch = y_train[idx]

      # Bias trick
      x_batch = np.hstack([x_batch, np.ones((x_batch.shape[0], 1))])

      # Weights update
      loss_val, grad = self.loss(x_batch, y_batch)
      self.W -= learning_rate*grad

      loss += loss_val
    return loss/(train_len)

  def loss(self,x, y):
    current_batch_size = x.shape[0]
    loss = 0.0
    dW = np.zeros(self.W.shape) # Grad
    for i in range(current_batch_size):
      scores = x[i].dot(self.W)
      correct_class_score = scores[int(y[i])]
      above_zero_loss_count = 0
      for j in range(self.classes_num):
        if j == y[i]:
          continue
        margin = scores[j] - correct_class_score + 1 # note delta = 1
        if margin > 0:
          above_zero_loss_count +=1
          loss += margin
          dW[:,j] += x[i] # We summarize it because grad computed over a batch and will de divided by number of examples in this batch
      dW[:,int(y[i])] -= above_zero_loss_count*x[i]
    loss /= current_batch_size
    dW /= current_batch_size
    return loss, dW
      
  def forward(self,x):
    x = np.append(x, 1)
    scores = x.dot(self.W)
    return np.argmax(scores) 


#### Не обязательное задание:
----
Поставлю дополнительно 20 баллов тому, кто графически оформит результаты этого эксперимента и напишет вывод о зависимости качества обучения от learning_rate и batch_size (возможно, нужно будет добавить варианты их сочетаний и/или увеличить количествоэпох до 20).

In [ ]:
print('Исследуем зависимость качества обучения от скорости:')

for lr in [1e-2, 1e-8]:
  for bs in [256, 2048]:

    print('-'*50, '\n', 'learning_rate =', lr, '\tbatch_size =', bs)
    print()
    lc_model = LinearClassifier(labels_ru, batch_size = bs)

    best_accuracy = 0
    for epoch in range(10):
      loss = lc_model.fit(x_train, y_train, learning_rate = lr) # исследуем зависимость качества обучения от скорости
      accuracy = validate(lc_model,x_test,y_test,noprint=True)
      if best_accuracy < accuracy:
        best_accuracy = accuracy
        best_epoch = epoch
      print(f"Epoch {epoch} \tLoss: {loss}, \tAccuracy:{accuracy}")

    print()
    print(f"Best accuracy is {best_accuracy} in {best_epoch} epoch")

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-change_weight.png" width="450"> 

https://docs.google.com/file/d/0Byvt-AfX75o1ZWxMRkxrUFJ2ZUE/preview

Идея в следующем: у нас есть некоторая поверхность, на которой будет точка минимума функции. Мы должны обновлять веса таким образом, чтобы смещаться в сторону этой точки. Loss показывает, как возрастает функция потерь. Так как потери - это плохо, мы должны их минимизировать, то есть мы должны уменьшать веса в сторону, обратную росту этой функции.

### 5.3 Выбор шага обучения

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-learning_rate.png" > 

Шаг обучения - некоторый коэффициент, как правило, небольшой, который не позволяет нам двигаться слишком быстро. 
У нас есть точка, в которую мы хотим попасть. Если мы сделаем слишком большой шаг, то мы ее перескочим (график справа), поэтому надо подобрать шаг, который позволит ее проскочить, но в то же время чтобы тот же процесс не шел слишком медленно (как на графике слева). 

Пока изменения loss функции достаточно большие, сам по себе градиент тоже большой. За счет этого можно двигаться быстро. Когда он будет уменьшаться, мы должны оказаться рядом с этим минимумом, и шаг, который мы выбрали, не должен мешать этому процессу. 

Бывают ситуации, когда шаг можно менять в самом процессе обучения, когда в начале обучения модели шаг большой, потом, по мере того, как она сходится, чтобы более четко найти минимум, шаг можно изменить вручную. Но изначально его нужно каким-то образом подобрать, и это зависит от данных и от самой модели. Это тоже гиперпараметр, связанный с обучением.

### 5.4 Выбор размера пакета



<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-gd.png" width="700"> 

Мы с самого начала говорили о пакетах. При этом было не очень понятно, чем они мотивированны. Точнее, мы мотивировали это тем, что у нас много данных и мы не сможем их обработать все, и это правда. Даже если мы сможем загрузить все данные в память, нам нужно будет загрузить их и использовать при рассчете в том числе градиента. Это ещё более затратно. 

Зачем он нужен при рассчете loss? Если мы посчитаем loss по одному изображению, скорее всего он будет очень специфичен, и это движение, которое произойдет, будет направлено в сторону минимума, потому что по этому конкретному изображению мы улучшим показатели.

Нас в первую очередь интересует более общее направление, которые будут работать на большинстве наших данных. Поэтому если мы будем оптимизировать модели, исходя из одного элемента данных, путь будет витиеватый, и процесс будет происходить достаточно долго. Если же данные не помещаются в память, то **можно использовать батч того размера, который у нас есть**. Можно загрузить в память и считать по этим мини-батчам градиент. В этом случае спуск будет более плавным, чем по одному изображению. 

Также при использовании всего датасета тоже есть свои минусы. **Не всегда загрузка всего датасета приводит к увеличению точности**. 

#### Важно !!

Подробнее: 
* https://habr.com/ru/post/318970/
* https://alxmamaev.medium.com/deep-learning-на-пальцах-часть-1-341cfe021ef9

### 5.5 Регуляризация

Наша модель может переобучаться - она может просто поднастроить веса по тренировочную выборку. Особенно это характерно для больших моделей, когда число весов приближается к числу параметров. 






<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier_regularisation.jpg" width="700"> 


Регуляризация Тихонова = L2 Regularization = weights decay

Идея состоит в том, что мы можем наложить некоторое требование на сами веса. Дело в том, что можно получить один и тот же выход модели при разных весах (выход модели соответствует умножению весов на $x$), при разных $w$ выход может быть идентичен. При этом если мы говорим о том, что эти параметры задают некоторую аппроксимацию нашей целевой функции, то аппроксимировать функцию можно двумя способами:
1. Использовать все имеющиеся данные и провести ее строго через все точки, которые нам известны; 
2. Использовать более простую функцию: (в данном случае линейную), которая не попадет точно во все данные, но зато будет соответствовать некоторым общим закономерностям, которые у них есть. 

Характерной чертой переобучения является второй сценарий, и сопровождается он, как правило большими весами. Введение L2-регуляризации приводит к тому, что большие веса штрафуются и предпочтение отдается решениям, использующим малые значения весов. 

Модель может попробовать схитрить и по-другому - использовать все веса, все признаки, даже незначимые, но с маленькими коэффициентами. С этим L2-loss поможет хуже, так как он не сильное штрафует мелкие веса. Результатов его применения - малые значения весов, которые использует модель

В этом случае на помощь приходит L1-loss, который штрафует вес за сам факт отличия его от нуля. Но и штрафует он все веса одинаково. Результат его применения - малое число весов, которые использует модель в принципе. 

<img src="https://edunet.kea.su/repo/src/L0X_Encoders/img/losses.gif" alt="alttext" style="width: 500px;"/>


Это лоссы можно комбинировать - получится Elastic Net


<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-regularisation2.png" width="700"> 


Берем сумму всех весов по всей матрице $w$, и добавляем ее к loss. Соответственно, чем больше будет эта сумма, тем больше будет суммарный loss. 


В дальнейшем проблема с переобучением будет вставать довольно часто. Методов регуляризации модели существует достаточно много. Этот — один из базовых, который будет использоваться практически во всех отпимизаторах. 

## 6. Cross-entropy Loss

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-12.jpg" width="650">

На этой части построены практически все модели, про которые будет идти речь.

Это оценка результата и обновление весов с использованием градиента. Для более глубоких моделей есть детали, как протолкнуть этот градиент в слои, которые находятся внутри. У нас один слой, поэтому мы достаточно легко посчитали от него градиент. Если слоев будет много, сделать это будет сложней.

Рассмотрим ещё одну loss-функцию. 

SVM loss хороша тем, что интуитивно понятно, как ее посчитать. Но проблема состоит в том, что модель выдает некоторые числа, которые сами по себе невозможно интерпретировать. Но сами по себе выходы модели мало что означают. Было бы неплохо, если бы мы сразу, глядя на выход модели для кошки, могли бы их как-то интерпретировать, не просматривая остальные веса. Хорошо бы, чтобы модель выдавала не какую-то абстрактную уверенность, а **вероятность** того, что по ее мнению, на картинке изображена кошка.


### 6.1 Переход к вероятностям

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier_softmax.jpg" width="750">

К этому можно придти, проведя с весами некоторые не очень сложные математические преобразования. 

На слайде выше показано, почему выходы модели часто называют logit’ами. Если предположить, что у нас есть некая вероятность, от которой мы берем такую функцию (logit), то она может принимать значения от нуля до бесконечности. Мы можем считать, что выходы модели - это logit’ы. 

Logit: https://en.wikipedia.org/wiki/Logit



<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier_softmax2.jpg" width="550">

А мы бы хотели получить не logit’ы, а настоящую вероятность на выходе модели. Для этого можно применить к сырым выходам несколько преобразований, поскольку вероятность должна быть неотрицательной, нам нужно сделать так, чтобы выходы модели тоже стали неотрицательными. Можно было бы их сдвинуть на на некоторое смещение, а можно сделать сложнее: провести над ними операцию экспоненцирования, то есть число Эйлера **возвести в степень**, соответствующую этому выходу. В результате мы получим вектор, уже гарантировано неотрицательных чисел (потому что мы ввели неположительное число в степень, пускай даже отрицательную. То есть выходы могут быть маленькие, но всегда положительные). 

Дальше, чтобы можно было интерпретировать эти числа как вероятности, их сумма должна быть равно нулю. Мы должны их нормализовать, то есть **поделить на сумму**. Это преобразование называется **Softmax функцией**.

**Получаются вероятности**, то есть числа, которые можно интерпретировать таким образом.

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier_cross-entropy.png" width="900">


Теперь, с использованием такого вектора можно определить другую loss функцию. Если не вникать в детали, можно **взять от нее логарифм**. Тогда loss от такого выхода будет выглядеть вот таким образом (формула на слайде выше), как нормализованный выход от верного класса на сумму остальных, к которому добавили минус. 

Получится график loss (слева). Его плюс заключается в том, что у него нет участка с плато, практически по всей длине функция получается гладкой, с хорошими мощными производными. Когда loss большой, производная тоже большая, и за счет этого можно быстро обучать модель. Для кусочно-линейной функции потерь она же равна константе.

До тех пор, пока модель не будет работать с точностью 100%, то есть пока все остальные выходы не станут нулевыми, мы можем продолжить обучение, даже если у нас нет явной регуляризации.

Осталось выяснить, почему такая простая на вид функция **называется так сложно** - Кросс-энтропия. 



### 6.2 Кросс-энтропия

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier_cross-entropy2.png" >

Если мы рассматриваем выходы нашей модели как вероятности, то мы можем их сравнивать. У нас есть номер правильного класса. Соответственно, можно сказать, что вероятность этого класса - единица, а всех остальных 0, и получить таким образом вероятностное распределение. Выход модели тоже можно интерпретировать как вероятности (тоже можно получить вероятностное распределение). И для работы с этими распределением есть некоторый математический аппарат, который основан на понятии энтропии, который ввел Клод Шеннон.


<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier_sun.png" >


Идея в следующем: у нас есть некоторые данные, которые мы передаем по каналу связи. Например, у нас есть метеостанция, которая сообщает прогноз погоды. Допустим, она может передавать 8 вариантов прогноза. Мы в каждый момент времени получаем от нее сообщение. Потребуется 3 бит, чтобы передавать это сообщение. 

Допустим, мы знаем некую вероятность, с которой в нашем регионе может наступить хорошая либо плохая погода. То есть мы знаем вероятностное распределение, по которыму у нас, допустим, солнечное место, где почти не бывает бури. Если мы знаем об этом, мы можем сэкономить на передаче информации. Мы можем закодировать наиболее вероятные сообщения двумя битами, причем придумать такие коды, чтобы они почти не пересекались. Они будут начинаться с нуля и пердавать 2 бита вместо трех. Таким образом можно сэкономить на передачи информации. То есть идея такая: **если мы знаем, что события маловероятны, мы можем кодировать их более длинной цепочкой, а более вероятные события - более короткими цепочками**. В этом случае в среднем количество информации, которое можно передать, сократиться.

**Формула Шеннона позволяет посчитать, насколько сильно мы можем сэкономить для конкретного вероятностного распределения**. То есть если подставить эти данные в формулу, то получим 2,23.

На практике реализовать это, используя биты, возможно, не выйдет, но это свойство данного вероятностного распределения можно посчитать. Оно несёт в себе некоторое количество полезной информации, соответствующее этой энтропии.

Что же такое кросс-энтропия?

Давайте представим, что мы каким-то образом начали кодировать эту информацию, прошили погодную станцию так, что она может эту кодированную информацию передавать, а пптотм перенесли в другой регион, где ситуация противоположная. И мы кодируем события, которые стали частыми, длинными цепочками, а маловероятные - короткой, и у нас в этом случае явно передается больше информации, чем мы могли бы, если бы сделаем все банально. 

Кросс-энтропия позволяет посчитать, насколько большая будет потеря в данном случае. То есть это некий **способ сравнить между собой вероятностные распределения**.

Что, собственно говоря, отображено в формуле, потому что мы считаем кросс-энтропию по нашему вектору P и Q. Вектор P состоит из нулей и единиц, соответственно, во всех случаях, кроме одного, вот это выражение будет нулевым, кроме одного случая с единицей. Случай с единицей соответствует нашему правильному классу. Поэтому сумма исчезает, остаются логарифм и вектор, а вероятность для правильного класса мы считаем как Soft max. Отсюда название кросс-энтропия.  


(В теории информации кросс-энтропия между двумя распределениями вероятностей p {\displaystyle p} и q {\displaystyle q} по одному и тому же базовому набору событий измеряет среднее число битов, необходимых для идентификации события , взятого из набора, если схема кодирования, используемая для набора, оптимизирована для оценочного распределения вероятностей q {\displaystyle q}, а не для истинного распределения p {\displaystyle p}.)

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier_cross-entropy3.png" width="800">

Аналогичное обоснование: есть понятие дивергенция, которая позволяет оценить расхождение между вероятностными распределениями, которая нам здесь и нужна. Есть дивергенция **Кульбака-Лейблера**, которая выражается через кросс-энтропию, а энтропия от нашего вектора нулевая. Поэтому фактически здесь кросс-энтропия равна дивергенции, которая показывает, насколько непохожи два распределения. 


<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier_softmax3.jpg" width="800">


Эта функция очень популярна, она используется при обучении реальных моделей на практике. Поскольку внутри находится Softmax, часто ее называют Softmax-классификатором, что, строго говоря, не совсем верно. Здесь функция потерь - кросс-энтропия, Softmax же просто используется внутри нее.


**Градиент Cross Entropy Loss**




http://machinelearningmechanic.com/deep_learning/2019/09/04/cross-entropy-loss-derivative.html


In [ ]:
n_classes = 10
y_class = 5

one_hot = np.zeros(n_classes)
one_hot[y_class] = 1
yi = one_hot
yi

$$ -L = \sum_i y_i \log p_i = \sum_i y_i \log(\dfrac {e^{s_{y_i}}} {\sum_j e^{s_{y_j}}})$$

$$s_{y_i} = w_i x$$

$$ \dfrac {\delta L} {\delta w_i} = \dfrac {\delta L} {\delta s_{y_i}} \dfrac {\delta s_{y_i}} {\delta w_i} $$

$$\dfrac {\delta s_{y_i}} {\delta w_i} = x$$

Только один $y_k = 1$


$$ -L = y_k \log p_i = \log(\dfrac {e^{s_{y_i}}} {\sum_j e^{s_{y_j}}})$$

i = k

$$ -L = \log(\dfrac {e^{s_{y_i}}} {\sum_j e^{s_{y_j}}}) = \log e^{s_{y_i}} - \log  \sum_j e^{s_{y_j}}  = s_{y_i} - \log  \sum_j e^{s_{y_j}}$$

$$\dfrac {\delta -L} {\delta s_{y_i}} = 1 - \dfrac 1 {\sum_j e^{s_{y_j}}} \cdot \dfrac {\delta {\sum_j e^{s_{y_j}}}} {\delta s_{y_i}} = 1 - \dfrac 1 {\sum_j e^{s_{y_i}}} \cdot \dfrac {\delta e^{s_{y_i}}} {\delta s_{y_i}} = 1 - \dfrac {e^{s_{y_i}}} {\sum_j e^{s_{y_j}}} = 1 - p_i$$

$$\dfrac {\delta L} {\delta s_{y_j}} = p_i - 1 $$

$$ \dfrac {\delta L_i} {\delta w_i}  = \dfrac {\delta L} {\delta s_{y_i}} \dfrac {\delta s_{y_i}} {\delta w_i} = (p_i - 1) x $$

i != k

$$ -L = \log(\dfrac {e^{s_{y_i}}} {\sum_j e^{s_{y_j}}}) = \log e^{s_{y_k}} - \log  \sum_j e^{s_{y_j}}  = s_{y_k} - \log  \sum_j e^{s_{y_j}}$$

$$\dfrac {\delta -L} {\delta s_{y_i}} = - \dfrac 1 {\sum_j e^{s_{y_j}}} \cdot \dfrac {\delta {\sum_j e^{s_{y_j}}}} {\delta s_{y_i}} =  \dfrac 1 {\sum_j e^{s_{y_i}}} \cdot \dfrac {\delta e^{s_{y_i}}} {\delta s_{y_i}} = \dfrac {e^{s_{y_i}}} {\sum_j e^{s_{y_j}}} = - p_i$$

$$\dfrac {\delta L} {\delta s_{y_j}} = p_i $$
$$ \dfrac {\delta L_i} {\delta w_i}  = \dfrac {\delta L} {\delta s_{y_i}} \dfrac {\delta s_{y_i}} {\delta w_i} = p_i x $$



**Практическое вычисление SoftMax**

Когда мы возводим число в степень, может получиться очень большое число и произойти переполнение. Неизвестно, какие будут у модели выходы. 

In [ ]:
f = np.array([123, 456, 789]) # example with 3 classes and each having large scores
p = np.exp(f) / np.sum(np.exp(f)) # Bad: Numeric problem, potential blowup
p

Поэтому мы можем вычесть из каждого $s_i$ пооложительную константу, чтобы уменьшить значения экспонент. В качестве константы можно выбрать максимальный элемент этого вектора, тогда у нас гарантированно не будет очень больших чисел и такой будет работать более стабильно.

$$M = \max_j s_{y_{j}}$$
$$s^{new}_{y_{i}}  = s_{y_{i}} - M $$

$$ \dfrac {e^{s^{new}_{y_{i}}}} {\sum_j e^{s^{new}_{y_{j}}}}  = \dfrac {e^{s_{y_{i}} - M }} {\sum_j e^{s_{y_{j}} - M }} = \dfrac {e^{s_{y_{i}}}e ^ {-M}} {\sum_j e^{s_{y_{j}}} e ^ {-M}} = \dfrac {e ^ {-M} e^{s_{y_{i}}}} {e ^ {-M} \sum_j e^{s_{y_{j}}} } = \dfrac { e^{s_{y_{i}}}} { \sum_j e^{s_{y_{j}}} }$$

In [ ]:
# instead: first shift the values of f so that the highest number is 0
f = np.array([123, 456, 789])
f -= f.max()
p = np.exp(f) / np.sum(np.exp(f))
f, p

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier_practice_softmax_code.png" width="800">


**Итог**

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier_itog.png" width="800">